In [46]:
from langchain_openai import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
import warnings
from langchain.chains import LLMChain,SimpleSequentialChain
from typing import Literal
from dotenv import load_dotenv
warnings.filterwarnings('ignore')
import os


 **Simple Invoking of LangChain ChatOpenAI**

In [2]:
load_dotenv()

True

In [3]:
llm=ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

In [4]:
response=llm.invoke("last date to train this model?")

In [5]:
print(response.content)

The training data for this model goes up until October 2021. If you're asking about updates or newer versions, I don't have information on specific release dates or future training schedules.


**USE of PromptTemplate and StrParser**

In [6]:
prompt=PromptTemplate.from_template("what is the capital of {country}?")

In [7]:
parser=StrOutputParser()

In [8]:
chain= prompt | llm | parser

In [9]:
response=chain.invoke({'country':'Pakistan'})

In [10]:
response

'The capital of Pakistan is Islamabad.'

**Real Time Chat application using langchain**

In [11]:
from logging import exception
chat=[]
while True:
  try:
    question=str(input("Enter your message!"))
    if (question=='Q'):
      break
    chat.append(HumanMessage(content=question))
    response=llm.invoke(chat)
    answer=parser.invoke(response)
    print('Question',chat[-1].content)
    print('Answer: ',answer)

    chat.append(AIMessage(content=response.content))

  except Exception as e:
    print(str(e))


Another way of using LLM chains

In [12]:
prompt=PromptTemplate.from_template("what is the capital of {country}?")


In [ ]:
model="gpt-4o-mini"

In [21]:
llm = ChatOpenAI(model_name=model, temperature=0) # type: ignore

In [29]:
response=LLMChain(llm=llm,prompt=prompt,output_parser=parser).invoke({'country':'Pakistan'})

In [30]:
response=LLMChain(prompt=prompt,llm=llm,output_parser=parser).invoke({'country':'Pakistan'})

In [31]:
response['text']

'The capital of Pakistan is Islamabad.'

Use of SimpleSequentialChain for calling on after another chain in sequence

In [33]:
prompt1=PromptTemplate.from_template("What is the capital of {country}?")
prompt2=PromptTemplate.from_template("Info regarding the {city}")

In [35]:
chain1=LLMChain(prompt=prompt1,llm=llm,output_parser=parser)
chain2=LLMChain(prompt=prompt2,llm=llm,output_parser=parser)

In [39]:
over_chain=SimpleSequentialChain(chains=[chain1,chain2])

In [40]:
response=over_chain.run("Italy")

In [41]:
response

'Rome, the capital of Italy, is a city rich in history, culture, and art. Here are some key points about Rome:\n\n1. **Historical Significance**: Rome is often referred to as the "Eternal City" and has a history that spans over 2,500 years. It was the center of the Roman Empire, which was one of the most powerful civilizations in history.\n\n2. **Cultural Heritage**: The city is home to numerous UNESCO World Heritage Sites, including the historic center of Rome, the Vatican City, and the ruins of ancient Rome, such as the Colosseum, the Roman Forum, and the Pantheon.\n\n3. **Vatican City**: Located within Rome, Vatican City is the smallest independent state in the world and serves as the spiritual and administrative center of the Roman Catholic Church. It is home to St. Peter\'s Basilica and the Sistine Chapel, famous for Michelangelo\'s ceiling.\n\n4. **Architecture**: Rome boasts a diverse range of architectural styles, from ancient Roman structures to Renaissance and Baroque masterp

Runnable parrallel allows two chains to run parallel

In [49]:
#all will remain just use runnable parralel
over_chain=RunnableParallel(country=prompt1 | llm,city=prompt2 | llm)

In [54]:
#we need to pass both parameter at once 
result=over_chain.invoke({'country':'Italy','city':'Rome'})

In [53]:
result['country'].content

'The capital of Italy is Rome.'